In [1]:
import pandas as pd
import numpy as np
import json
import re
from pyathena import connect
from pyathena.pandas.util import as_pandas
from pyathena.pandas.cursor import PandasCursor
import multiprocessing
import concurrent

In [2]:
client_df = pd.read_excel("./Tobii underlag.xlsx", converters={"Placement Id": int})
client_placement_ids = list(client_df["Placement Id"].unique())

In [3]:
# Fetch impressions using PyAthena library
cursor = connect(s3_staging_dir="s3://adhoc-athena-output-bucket-eu-north-1/",
                 region_name="eu-north-1", profile_name="atexprodadminsso", cursor_class=PandasCursor).cursor()

impressions_df = cursor.execute('''
SELECT 
    "impression_id",
    "date",
	"is_fixated",
	"is_iab_inview",
	"hostname",
	array_distinct(
		regexp_extract_all(
			json_format(
				cast(
					transform(
						html_metadata,
						x->ROW(x."inline_scripts", x."non_standard_attributes")
					) as JSON
				)
			),
			'tag_id.{0,10}?(?:%[0-9A-Fa-f]{2}){0,5}?([\d]+)',
			1
		)
	) as placement_ids,
	CONCAT(
		cast(width as VARCHAR),
		'x',
		cast(height as VARCHAR)
	) AS size,
	"exist_viewable_1_s_threshold_50",
	"exist_viewable_2_s_threshold_50"
FROM "data_pipelines_storage_semarket_prod"."extension_impression"
WHERE extract(year from date) = 2023
''').as_pandas()

In [4]:
impressions_df.impression_id.count()

13900386

In [5]:
def extract_pid(placement_ids):
    placement_ids = json.loads(placement_ids)
    return (
        placement_ids[0] if placement_ids else None
    )

def pid_type(placement_id):
    try:
        int(placement_id)
        return "int"
    except ValueError:
        return "str"

impressions_df["pid"] = impressions_df["placement_ids"].apply(lambda x: extract_pid(x)).astype(np.object_)
impressions_df["pid_type"] = impressions_df["pid"].apply(lambda x: pid_type(x))
impressions_df = impressions_df[impressions_df["pid"].notna()]

In [6]:
grouped_df = impressions_df.groupby(["pid", "size", "hostname"]).apply(
    lambda x: pd.Series(
        {
            "impressions": x["impression_id"].count(),
            "fixations": x.loc[x["is_fixated"], "impression_id"].count(),
            "inview": x.loc[x["is_iab_inview"], "impression_id"].count()
        }
    )
)
grouped_df = grouped_df.reset_index()

In [7]:
grouped_df["fixation ratio %"] = round(grouped_df["fixations"] / grouped_df["impressions"] * 100, 2).fillna(0).replace([np.inf, -np.inf], 0)
grouped_df["inview ratio %"] = round(grouped_df["inview"] / grouped_df["impressions"] * 100, 2).fillna(0).replace([np.inf, -np.inf], 0)
grouped_df["fixation/inview ratio %"] = round(grouped_df["fixations"] / grouped_df["inview"] * 100, 2).fillna(0).replace([np.inf, -np.inf], 0)
grouped_df

,pid,size,hostname,impressions,fixations,inview,fixation ratio %,inview ratio %,fixation/inview ratio %
0,0.000000e+00,560x315,nbcnews.com,1,0,1,0.0,100.0,0.0
1,3.000000e+00,0x0,brinkenbakar.se,18,0,0,0.0,0.0,0.0
2,3.000000e+00,0x0,convertcsv.com,1,0,0,0.0,0.0,0.0
3,3.000000e+00,0x0,einthusan.tv,5,0,0,0.0,0.0,0.0
4,3.000000e+00,0x0,gameclubz.com,3,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
24965,3.038301e+18,1536x714,se.boohoo.com,1,0,1,0.0,100.0,0.0
24966,3.038301e+18,1707x843,se.boohoo.com,1,0,1,0.0,100.0,0.0
24967,3.038301e+18,1920x969,se.boohoo.com,1,0,1,0.0,100.0,0.0
24968,3.097723e+18,1280x563,westernunion.com,2,0,2,0.0,100.0,0.0


In [8]:
merged_df = pd.merge(client_df, grouped_df, left_on=["Placement Id", "Size", "Hostname"], right_on=["pid", "size", "hostname"], how="inner")
merged_df

,Seller Name,Publisher Name,Placement Id,Size,Hostname,pid,size,hostname,impressions,fixations,inview,fixation ratio %,inview ratio %,fixation/inview ratio %
0,BONNIER NEWS AB,BNS Expressen,20148635,970x250,expressen.se,20148635.0,970x250,expressen.se,56,30,34,53.57,60.71,88.24
1,BONNIER NEWS AB,BNS Expressen,20148635,980x120,expressen.se,20148635.0,980x120,expressen.se,2,0,2,0.00,100.00,0.00
2,BONNIER NEWS AB,BNS Expressen,20148635,980x240,expressen.se,20148635.0,980x240,expressen.se,203,53,104,26.11,51.23,50.96
3,BONNIER NEWS AB,BNS Expressen,20148635,980x300,expressen.se,20148635.0,980x300,expressen.se,30,8,16,26.67,53.33,50.00
4,BONNIER NEWS AB,BNS Expressen,20148635,980x360,expressen.se,20148635.0,980x360,expressen.se,5,2,1,40.00,20.00,200.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,Stampen Lokala Medier AB,Unexposed Publisher,14444864,640x360,hn.se,14444864.0,640x360,hn.se,8,6,8,75.00,100.00,75.00
976,Stampen Lokala Medier AB,Unexposed Publisher,14444868,640x360,hn.se,14444868.0,640x360,hn.se,2,1,1,50.00,50.00,100.00
977,Stampen Lokala Medier AB,Unexposed Publisher,14444868,980x240,hn.se,14444868.0,980x240,hn.se,1,1,1,100.00,100.00,100.00
978,Stampen Lokala Medier AB,Unexposed Publisher,14444868,980x600,hn.se,14444868.0,980x600,hn.se,1,0,0,0.00,0.00,0.00


In [9]:
merged_df.sort_values("impressions", ascending=False)

,Seller Name,Publisher Name,Placement Id,Size,Hostname,pid,size,hostname,impressions,fixations,inview,fixation ratio %,inview ratio %,fixation/inview ratio %
693,Schibsted Sweden,SE-Aftonbladet,19499441,980x240,aftonbladet.se,19499441.0,980x240,aftonbladet.se,45919,6506,33648,14.17,73.28,19.34
695,Schibsted Sweden,SE-Aftonbladet,19499442,250x600,aftonbladet.se,19499442.0,250x600,aftonbladet.se,39771,2474,35816,6.22,90.06,6.91
680,Schibsted Sweden,SE-Aftonbladet,19499428,640x320,aftonbladet.se,19499428.0,640x320,aftonbladet.se,33161,4051,13164,12.22,39.70,30.77
666,Schibsted Sweden,SE-Aftonbladet,19499420,300x600,aftonbladet.se,19499420.0,300x600,aftonbladet.se,31409,2221,12493,7.07,39.78,17.78
697,Schibsted Sweden,SE-Aftonbladet,19499443,250x600,aftonbladet.se,19499443.0,250x600,aftonbladet.se,25044,1011,12667,4.04,50.58,7.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,LEEADS AB,Friidrott,21515901,980x240,friidrott.se,21515901.0,980x240,friidrott.se,1,0,1,0.00,100.00,0.00
447,LEEADS AB,Garaget,21162399,300x600,garaget.org,21162399.0,300x600,garaget.org,1,0,0,0.00,0.00,0.00
800,Schibsted Sweden,SE-SvD,19499699,300x250,svd.se,19499699.0,300x250,svd.se,1,0,1,0.00,100.00,0.00
446,LEEADS AB,Svenska Lag,17867855,300x250,svenskalag.se,17867855.0,300x250,svenskalag.se,1,0,1,0.00,100.00,0.00


In [11]:
merged_df.to_excel("omd_performance_2023.xlsx", index=False)

In [13]:
merged_df.count()

Seller Name                980
Publisher Name             980
Placement Id               980
Size                       980
Hostname                   980
pid                        980
size                       980
hostname                   980
impressions                980
fixations                  980
inview                     980
fixation ratio %           980
inview ratio %             980
fixation/inview ratio %    980
dtype: int64